# Домашнее задние к уроку №4 "Система управления базами данных MongoDB в Python"

1 (+). Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии в вашу базу.

2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

3. * Любая аналитика. Например matching ваканский с разных площадок

In [1]:
from pymongo import MongoClient
import json

In [2]:
client = MongoClient('mongodb://localhost:27017')

In [91]:
#client.list_database_names()

Создаём базу данных "Vacancies" с коллекцией "collect_vacancies"

In [4]:
db = client.vacancies
collection_v = db.collect_vacancies

Загружаем данные из файла созданного при парсинге

In [92]:
with open('Вакансии.json', mode='r', encoding='UTF-8') as obj:
    dat = json.load(obj)

Загружаем данные в коллекцию

In [25]:
for d in dat:
    collection_v.insert_one(d)

In [180]:
vc = collection_v.find()

In [189]:
vc.next()

{'_id': ObjectId('633db6e4611d7fa103a8ebf0'),
 'name': 'Наладчик-оператор (Снековый завод)',
 'companay': 'ООО\xa0PepsiCo',
 'place': 'Новосибирск',
 'salary': {'string': 'от 45\u202f000 руб.',
  'from': 45000,
  'to': None,
  'valute': 'руб'},
 'link': 'https://tver.hh.ru/vacancy/70411190?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%9D%D0%B0%D0%BB%D0%B0%D0%B4%D1%87%D0%B8%D0%BA',
 'source': 'hh.ru'}

Фунция для добавления новых вакансий

In [166]:
def append_vacancy(c, v):
    # считаем, что вакансия повторяется, если у неё совпадают наименование компании и наименование вакансии
    # если вакансия добавлена, то функция возвращает True, если вакансия уже есть в базе, то функция возвращает False
    r = c.count_documents({'name' : v['name'], 'companay': v['companay']})
    if r:
        return False
    else:
        c.insert_one(v)
        return True

функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы

In [119]:
def find_greate_vacancies(c, border):
    res = c.find({'$and': 
                           [ 
                            {'salary': {'$ne': None}},
                            {'$or': 
                                [{'$and': 
                                    [{'salary.from': None},
                                     {'salart.to': {'$gte': z}}]
                                 },{'$and':
                                    [{'salary.to': None},
                                     {'salary.from': {'$gte': z}}]
                                 },{'salary.to': {'$gte': z}
                                    
                                 } 
                                ]
                            }
                           ]
                      })
    for v in res:
        print(f" {v['companay']} \t - \t {v['name']}  {v['salary']['from']} - {v['salary']['to']} {v['salary']['valute']}")

In [120]:
find_greate_vacancies(collection_v, 100_000)

 АО ОЧАКОВО 	 - 	 Оператор-наладчик производственной линии  None - 150000 руб
 ООО Феррополис 	 - 	 Наладчик токарного станка с ЧПУ  95000 - 150000 руб
 ООО Рогос 	 - 	 Токарь - наладчик станков с ЧПУ  180000 - None руб
 Аигма 	 - 	 Наладчик термоформовочного оборудования  100000 - 150000 руб
 Логинова Ольга Ивановна 	 - 	 Механик-наладчик оборудования на обогатительный комбинат  250000 - 300000 руб
 ТОО BRAMF 	 - 	 Мастер-наладчик станков (мебельное производство)  400000 - None KZT
 ООО НПЦ ЛКТ 	 - 	 Оператор-наладчик станков с ЧПУ  70000 - 150000 руб
 ООО Альпы 	 - 	 Инженер-наладчик (Холодильное оборудование)  250000 - None руб
 ИП Молдагазинова Назира 	 - 	 Наладчик оборудования Tetra Pak.  300000 - 500000 KZT
 ИП Сулейменова Лейла 	 - 	 Инженер-наладчик КИПиА  400000 - 500000 KZT
 AP-UNITYHR 	 - 	 Наладчик (оператор) станков с ЧПУ  100000 - 150000 руб
 ООО Инженерные системы и технологии 	 - 	 Наладчик вентиляционного оборудования  50000 - 200000 руб
 ИП Молдагазинова Назира 	 - 	